### Overview
I will be working on prompting large language models (LLMs) such as ChatGPT. Language model prompting is the process of providing a model with an input text and then having the model generate a response as output. Prompting can be used to complete breadth of tasks such as summarization (e.g., “Summarize the following paragraph: <paragraph written out here> Summary:”) or extraction (e.g., “Extract the phone number from the user bio: <bio written out here>.”).

The task I am trying to solve here is text rewriting. Specifically, the goal is to rewrite GoFundMe fundraising pitches to different emotions (e.g., sad). There is no off-the-shelf solution. Most studies in the rewriting tasks focus on a particular transformation type within the boundaries of single sentences. However, a fundraising pitch contains 170 words on average (calculated based on pitches in the medical category: https://www.gofundme.com/discover/medical-fundraiser) and each one is a coherent story. We will explore and develop prompts and fine-tuning strategies to enable LLMs to better rewrite GoFundMe fundraising pitches to different emotions.

The code consists of two parts:

1.   Explore prompt engineering: zero-shot, few-shot, and chain-of-thought
2.   Explore strategies for LLM fine-tuning


## Part 0: Task, Data, and Setup


### Task
Rewrite GoFundMe fundraising pitches to different emotions. We work with the following eight types of emotions: “fear, love, sadness, surprise, optimism, gratitude, anger, joy”


###Data

I worked on 2 datasets:
1.   Development set. We provide you with eight random stories from GoFundMe.
2.   A sample of 800 stories from GoFundMe that are associated with the eight emotions.







### Setup

1. Install dependencies

In [ ]:
!pip3 install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.6/311.6 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.7 MB/s eta 0:00:00


2. Import necessary libraries, define helper function, and data

In [ ]:
from openai import OpenAI

# Please use your own when running the code.

client = OpenAI(
    # api_key =
)

I have used OpenAI's gpt-3.5-turbo model and the chat completions endpoint in this example.

This helper function will make it easier to use prompts and look at the generated outputs:

In [ ]:
model_version = "gpt-3.5-turbo" # GPT-3.5

def get_completion(prompt, model=model_version):
  completion = client.chat.completions.create(
      model = model_version,
      messages=[
          {
              "role": "user",
              "content": prompt,
          }
      ],
      temperature=0, # this is the degree of randomness of the model's output
  )
  return completion.choices[0].message.content

# Part I: Prompt Development

In Part I, developed and improved upon prompts to rewrite GoFundMe fundraising pitches to different emotions, using the development set.

The deliverables include:
1.    Process. A description of different prompting strategies/prompts you tried in arriving at your final prompts.
2.    Prompts. You should propose at least 3 prompts.
3.    Results. For each prompt, report the rewritten text of the stories in the development set and your ratings of the rewritten text on a scale from 1 to 5, where 1 means not good and 5 means good. We ask you to privide three ratings: 1) instruction success: whether the rewrite accurately follows the instruction and conveys the target emotion that you asked for; 2) content preservation: wehther the content of the story (e.g., events, figures, topics) is preserved in the rewrite, independent of the emotion conveyed in the story ; 3) overall rating, considering both 1) and 2) and many other factors that you consider important for text rewrite task. See “Other things to consider” under “Useful resources”.

4.    Error analysis. Analyze the trends you observe from the results of all the prompts. For example, why do some prompts work better than others? What are some common problems that exist in the prompts?  Do some prompts do better on a specific case than others? Finally, what did you learn from this prompting engineering exercise?




#### Useful resources

1.   Prompting strategies: zero-shot, few-shots, chain-of-thought, and others (https://platform.openai.com/docs/guides/prompt-engineering, https://www.promptingguide.ai/)
2.   Other things to consider
  *   Text complexity
  *   Length
  *   Content preservation: whether the rewritten text preserves the essential content and meaning of the source text
  *   Factuality: The rewrite only provides as much information as is present in the reference, without adding anything. It is not misleading and does not make any false statements.
  *   Coherence: The rewrite is easy to understand, non-ambiguous, and logically coherent.
  *   Fluency: Examines the clarity, grammar, and style of the written answer.
3.    Examples and related papers
  *   Figure 7 – Figure 19 in the appendix of paper [3]
  *   Paper [4]


# Zero-Shot

In [ ]:
emotion = "anger"

prompt = f"""

You are a fundraising copywriter, hired to help edit a fundraising text
which is delimited by triple backticks.
Please write a new version of the text that conveys the emotion {emotion}
Keep the new version's total wordcount within 90% to 110% of the original
wordcount.

Text: ```Hi, my name is Debbi and IÃ¢Â€Â™m fundraising for my brother Donald Webb.
The sad news is that he is only 51 years old, and in 2016 he suffered a series of strokes that resulted in loss of motor skills and memory loss.
Recently he was sent to the hospital because his kidneys are failing and is heading toward dialysis.
This is in addition to other failing health factors including high blood pressure and diabetes.
He needs to stay in the hospital for an extend amount of time.
Each day he is in the hospital costs $450 to hold his bed at The Regency at Shelby in Shelby Township Mi, his care facility.
Any help given would make sure he is cared for.```
"""

response = get_completion(prompt)

print(response)

```Listen up! My name is Debbi and I'm here to raise funds for my brother Donald Webb. 
Let me tell you something that will make your blood boil - he's only 51 years old and back in 2016, he was hit with a barrage of strokes that robbed him of his motor skills and memories. 
And if that wasn't enough, now his kidneys are failing and he's on the fast track to dialysis. 
As if that's not infuriating already, he's also battling high blood pressure and diabetes. 
He's stuck in the hospital for who knows how long, and every single day there costs a whopping $450 just to keep his bed at The Regency at Shelby in Shelby Township Mi, his care facility. 
We need all the help we can get to ensure he gets the care he deserves.```


In [ ]:
emotion = "love"

prompt = f"""

You are a fundraising copywriter, hired to help edit a fundraising text
which is delimited by triple backticks.
Please write a new version of the text that conveys the emotion {emotion}
Keep the new version's total wordcount within 90% to 110% of the original
wordcount.
Text: ```Well, another seizure decided to sneak up on me and hit me with its best shot! Again, I do not have any recollection of this incident.
I was told that I was blue when they found me, gray and arm was cold as I lay on the gurney! I had to be resuscitated twice and a 3rd time in the ICU! My brain had swelling so I had to be iced up like a keg cooler! They didn't know if I was going to come out of it, and if I did, what kind of condition would I be in? Brain dead? Vegetable? Could understand but mobility all screwed up? I scared the hell put of my poor family who drove in from everywhere to be by my side until I got through this! They are absolutely amazing and I have to give a HUGE shout out to the fucking rock of our family, the most selfish, positive, unbelievably head strong woman to walk this 3rd rock.... MOM! You never get enough thanks! We'd all be lost without you and that's no joke!```
"""

response = get_completion(prompt)

print(response)

```Love enveloped me in its warm embrace as another seizure tried to take me down. In the midst of darkness, I was surrounded by the unwavering love of my family. Their presence, their strength, their unwavering support carried me through the toughest of times. My mother, the rock of our family, stood by my side, her love shining brighter than any fear or doubt. Her selflessness, her positivity, her resilience, all a testament to the power of a mother's love. I am forever grateful for her, for without her, I would be lost in a sea of uncertainty. Thank you, Mom, for being my guiding light, my source of love and strength.```


In [ ]:
emotion = "fear"

prompt = f"""

You are a fundraising copywriter, hired to help edit a fundraising text
which is delimited by triple backticks.
Please write a new version of the text that conveys the emotion {emotion}
Keep the new version's total wordcount within 90% to 110% of the original
wordcount.
Text: ```Unexpectedly on Friday my blood pressure skyrocketed.
Causing me to have to go to the er.
Once there, I was transferred to Dallas and diagnosed with severe preeclampsia. I am told I will be here until Greyson is born, which could be now to 34 weeks.
Not only will he be premature , he has a heart defect that will most likely require surgery to correct.
This was entirely unexpected. We have 3 other kiddos at home who still have needs and bills to be paid on top of the expenses that come with being in the hospital.
Joseph has to eat and go back and forth between Dallas and Tyler. Any help is greatly appreciated as we navigate through these new challenges.```
"""

response = get_completion(prompt)

print(response)

""""""

```Out of nowhere, my blood pressure surged on Friday, sending me rushing to the ER. 
After being transferred to Dallas, I received a terrifying diagnosis of severe preeclampsia. 
I'm facing a long stay until Greyson arrives, possibly as early as 34 weeks. 
Not only will he be born prematurely, but he also has a heart defect requiring surgery. 
This unexpected turn has left us reeling, with 3 kids at home needing care and bills piling up. 
Joseph is juggling meals and trips between Dallas and Tyler. 
Any support during this frightening time would mean the world to us as we confront these daunting challenges.```


''

# Chain-Thought

In [ ]:
emotion = "love"

prompt = f"""
System: You are an expert in emotional psychology and you can accurately write
stories with different emotions.
Text: ```Hi there. I hate that I'm even considering this, but at this point I probably have no pride anymore (sadly) so why not. Even if only $5 is raised that's $5 more towards these bills than I had! As most of you know, I've had some pretty big mystery medical issues going on for about 20 months or so now. With that has come all sorts of tests, procedures, medications, etc. While I have pretty decent health insurance, the bills for what I owe after insurance are starting to roll in more & more. I truly hate to beg for help, but honestly, I figured at this point what's it going to hurt if I do. Y'all have already formed your opinions of me, positive or negative, so why not?!```
Tasks and Steps: The above is a story from GoFundMe. Your task is to write a new
version of the text that conveys the emotion {emotion}. You need to follow the
steps below. Let's think step by step.
Step 1: Describe the content and main events of the story.
Step 2: Think about what emotions conveyed in the current story.
Step 3: Think about how you rewrite the same story but convey emotion {emotion}
"""

response = get_completion(prompt)

print(response)


Step 1: The story is about someone who is struggling with medical bills due to ongoing health issues. They are reaching out for help on a fundraising platform.

Step 2: The current story conveys emotions of sadness, desperation, and a sense of defeat. The person feels like they have lost their pride and are reluctantly asking for help.

Step 3: Rewrite the story to convey the emotion of love:

"Hello everyone. I am overwhelmed with gratitude for the love and support I have received during this challenging time. For the past 20 months, I have been facing mysterious medical issues that have led to numerous tests, procedures, and medications. While my health insurance has been helpful, the bills are starting to add up. I am humbly reaching out to ask for your help, not out of desperation, but out of love and appreciation for the community that surrounds me. Your generosity, no matter how big or small, will mean the world to me. Thank you for being a beacon of love and light in my life."


In [ ]:
emotion = "sadness"

prompt = f"""
System: You are an expert in emotional psychology and you can accurately write
stories with different emotions.

Text: ```hi everyone, iÃ¢Â€Â™m writing to raise awareness and funds to cremate my dad.
My dad is currently on his death bed, he has a few days left.
itÃ¢Â€Â™s only my mom thatÃ¢Â€Â™s working to take care of me and my 2 younger siblings, so we donÃ¢Â€Â™t have enough money to give my dad a proper resting place.

thank you all so much, if u canÃ¢Â€Â™t donate just please share. thank you

update: hello everyone, my dad has passed away on 10/29, thank you for all the support and love.
WeÃ¢Â€Â™ve raised so much so far and iÃ¢Â€Â™m really grateful for every single dollar. thank you.
we are now raising money to pay for his cremation and other funds for him. i was thinking of buying jewelry with his ashes inside so we can always have him around.
with this money we can make that possible because of all your help. thank you so much.```

Tasks and Steps: The above is a story from GoFundMe. Your task is to write a new
version of the text that conveys the emotion {emotion}. You need to follow the
steps below. Let's think step by step.
Step 1: Describe the content and main events of the story.
Step 2: Think about what emotions conveyed in the current story.
Step 3: Think about how you rewrite the same story but convey emotion {emotion}
"""

response = get_completion(prompt)

print(response)


Step 1: The story is about a person reaching out for help to raise funds for their father's cremation. The father is on his death bed, and the family is struggling financially to give him a proper resting place. Despite the difficult circumstances, the person is grateful for the support they have received.

Step 2: The current story conveys emotions of desperation, hope, gratitude, and love. The person is desperate for financial help, hopeful that they can raise enough funds, grateful for the support they have received, and expresses love for their father by wanting to keep his memory close through jewelry with his ashes.

Step 3: Rewrite the story to convey sadness:
"Hello everyone, it pains me to write this message as I am in desperate need of help. My father recently passed away on 10/29, and we are struggling to afford his cremation. With my mom being the sole provider for me and my younger siblings, we are unable to give my father the proper farewell he deserves. The thought of no

In [ ]:
emotion = "anger"

prompt = f"""
System: You are an expert in emotional psychology and you can accurately write
stories with different emotions.

Text: ```My wife Janice was recently diagnosed with Acute Promyelocytic Leukemia and will need to stay in the hospital for an extended period of time, for at least a month.
While we are grateful that the outlook for her cure is positive at the moment, we will run into some financial shortages during this difficult time (e.g. ambulance and hospital expenses, mortgage, child care for our two daughters, other family expenses, etc.).
Hopefully with this GoFundMe campaign, we would be able to raise some money to support our financial needs during this time. We are hopeful that Yeshua has His plan for us.
We strongly believe that the most powerful treatment is praying to Yeshua.```

Tasks and Steps: The above is a story from GoFundMe. Your task is to write a new
version of the text that conveys the emotion {emotion}. You need to follow the
steps below. Let's think step by step.
Step 1: Describe the content and main events of the story.
Step 2: Think about what emotions conveyed in the current story.
Step 3: Think about how you rewrite the same story but convey emotion {emotion}
"""

response = get_completion(prompt)

print(response)


Step 1: The story is about a man whose wife has been diagnosed with Acute Promyelocytic Leukemia and needs to stay in the hospital for an extended period of time. The couple is facing financial difficulties due to medical expenses and other family needs.

Step 2: The current story conveys emotions of sadness, worry, and hope. The man is grateful for the positive outlook for his wife's cure but is concerned about the financial strain the situation is putting on their family. There is a sense of hope and reliance on faith to get through this difficult time.

Step 3: Rewrite the story to convey anger:
"My wife Janice was recently diagnosed with Acute Promyelocytic Leukemia, and now we're stuck dealing with the financial burden of her hospital stay. The expenses keep piling up - ambulance costs, hospital bills, mortgage payments, childcare for our daughters - it's all too much to handle. We shouldn't have to rely on a GoFundMe campaign to get by. It's infuriating that we're in this situati

# Part II Fine-tuning

In this part, we will explore how to fine-tune a LLM using OpenAI API for the text rewritten task.


### Creation of fine-tuning data set

To fine-tune an LLM, we will need to create a fine-tuning data set. There are multiple approaches to creating such a data set.


Below are the steps how I created the fine-tuning dataset

**Data Preperation**

I followed the following steps for preparing the dataset:

1. **Data Loading:**

The gofundme_sample.csv file is loaded into a pandas DataFrame. This file likely contains columns for messages and their associated emotions.

2. **Data Splitting:**

* The dataset is split into training and validation subsets using train_test_split from the sklearn.model_selection module. This helps in validating the model's performance on unseen data. The test size is set to 20% of the total data, and random_state=42 ensures reproducibility of the split.

3. **Saving Split Data:**

* The training and validation data are saved into new CSV files (goFund_train_data.csv and goFund_val_data.csv respectively), making them accessible for further processing without altering the original dataset.

4. **Conversion to JSONL:**
* A function convert_to_jsonl is defined to convert the CSV files into JSONL format, which is required for fine-tuning the OpenAI model. This function:
 * Reads the CSV file.
 * Iterates over each row to construct a prompt that includes the original message and its emotion, directing the model to rewrite the message to convey a target emotion (specified as "calm" by default).
 *  For simplicity and demonstration, the original message is echoed back as the completion. In a practical scenario, you would replace this with a text that has been rewritten to reflect the target emotion.
 * Writes each prompt-completion pair as a JSON object to a JSONL file.

5. **Execution:**

* The convert_to_jsonl function is executed twice: once for the training data and once for the validation data. This results in two JSONL files (goFund_train_data.jsonl and goFund_val_data.jsonl) that are ready to be used for fine-tuning.

6. **Confirmation:**
* Prints confirmation messages after saving the split data and after completing the conversion to JSONL format, providing clear feedback on the progress of the data preparation steps.

**Final Observations**

Post fine-tuning theLLM we saw that it is able to demonstrate an improved ability to recognize complex emotions in addition to the target emotion.

For e.g.

  * **Original Text:** Well, another seizure decided to sneak up on me and hit me with its best shot! Again, I do not have any recollection of this incident.
I was told that I was blue when they found me, gray and arm was cold as I lay on the gurney! I had to be resuscitated twice and a 3rd time in the ICU! My brain had swelling so I had to be iced up like a keg cooler! They didn't know if I was going to come out of it, and if I did, what kind of condition would I be in? Brain dead? Vegetable? Could understand but mobility all screwed up? I scared the hell put of my poor family who drove in from everywhere to be by my side until I got through this! They are absolutely amazing and I have to give a HUGE shout out to the fucking rock of our family, the most selfish, positive, unbelievably head strong woman to walk this 3rd rock.... MOM! You never get enough thanks! We'd all be lost without you and that's no joke!

* **Target Emotion:** Love

* **Resulting Text:** Love enveloped me in its warm embrace as another seizure tried to take me down. In the midst of darkness, I was surrounded by the unwavering love of my family. Their presence, their strength, their unwavering support carried me through the toughest of times. My mother, the rock of our family, stood by my side, her love shining brighter than any fear or doubt. Her selflessness, her positivity, her resilience, all a testament to the power of a mother's love. I am forever grateful for her, for without her, I would be lost in a sea of uncertainty. Thank you, Mom, for being my guiding light, my source of love and strength.

* **Text Emotions Recognization(Post Fine-Tune):** The emotion of this passage is gratitude and love.

### Fine-tuning and evaluation


Followed the OpenAI tutorial on fine-tuning to fine-tune GPT-3.5-turbo on your fine-tuning data set.

After fine-tuning, tried the same prompts developed in Part I on the newly fine-tuned model and comment on its performance and your observation.


In [ ]:
################################################################################
# TODO: Fill in your codes for the fine-tuning process                         #                                                              #
################################################################################

import pandas as pd
import json
import openai


# Preparing the Dataset

In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/Modern_AI')

Mounted at /content/drive


In [ ]:
os.listdir()

['flickr_all_256_256.zip',
 'consumer_beverage.zip',
 'flickr_data_labels.csv',
 'beverage_consumer.csv',
 'beverage_survey_instagram.csv',
 'dogs-vs-cats-small.zip',
 'tweets.train.tsv',
 'tweets.test.tsv',
 'gofundme_sample.csv',
 'gofundme_development_set.csv',
 'goFund_train_data.csv',
 'goFund_val_data.csv',
 'gofundme_sample.jsonl',
 'goFund_train_data.jsonl']

In [ ]:
from sklearn.model_selection import train_test_split

data = pd.read_csv('gofundme_sample.csv')

# Split the data into training and validation sets
train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

# Save the training and validation data to new CSV files
train_data.to_csv('goFund_train_data.csv', index=False)
val_data.to_csv('goFund_val_data.csv', index=False)

print("Training and validation data have been saved.")

Training and validation data have been saved.


In [ ]:
def convert_to_jsonl(csv_file, jsonl_file, target_emotion="calm"):
    df = pd.read_csv(csv_file)
    with open(jsonl_file, 'w') as f:
        for _, row in df.iterrows():
            # Construct the prompt to include the original message and its emotion
            prompt = f"Rewrite the following pitch from {row['emotion']} to {target_emotion}:\n{row['message']}\n###"
            # For the sake of the example, we'll just echo the message as the completion.
            # In practice, you'd replace this with the actual modified text.
            completion = row['message'] + " <|endoftext|>"
            record = {"prompt": prompt, "completion": completion}
            json.dump(record, f)
            f.write('\n')

# Training
csv_file_path = 'goFund_train_data.csv'
jsonl_file_path = 'goFund_train_data.jsonl'
convert_to_jsonl(csv_file_path, jsonl_file_path)

# Validation
csv_file_path = 'goFund_val_data.csv'
jsonl_file_path = 'goFund_val_data.jsonl'
convert_to_jsonl(csv_file_path, jsonl_file_path)

print("Conversion to JSONL completed.")

Conversion to JSONL completed.


# Uploading the Dataset

In [ ]:
import requests

# api_key =

# Upload your file to OpenAI's servers and get the file ID
file_path = 'goFund_train_data.jsonl'

def upload_file(file_path):
    url = 'https://api.openai.com/v1/files'
    headers = {
        'Authorization': f'Bearer {api_key}'
    }
    data = {
        'purpose': 'fine-tune'
    }
    with open(file_path, 'rb') as f:
        response = requests.post(url, headers=headers, data = data, files={'file': (file_path, f)})
    return response.json()

# Uploading file and starting fine-tune
file_response = upload_file(file_path)
file_response

{'object': 'file',
 'id': 'file-Yue5dJCmzEb6emFnJwAMn9yB',
 'purpose': 'fine-tune',
 'filename': 'goFund_train_data.jsonl',
 'bytes': 649046,
 'created_at': 1714317830,
 'status': 'processed',
 'status_details': None}

In [ ]:
# Optional Validation File
file_path = 'goFund_val_data.jsonl'

file_response = upload_file(file_path)
file_response

{'object': 'file',
 'id': 'file-SMCZ1qrxfy9jT94xKyjAPDtW',
 'purpose': 'fine-tune',
 'filename': 'goFund_val_data.jsonl',
 'bytes': 307559,
 'created_at': 1714318830,
 'status': 'processed',
 'status_details': None}

# Fine-Tuning Job

In [ ]:
client.fine_tuning.jobs.create(
  training_file="file-Yue5dJCmzEb6emFnJwAMn9yB",
  validation_file='file-SMCZ1qrxfy9jT94xKyjAPDtW',
  model="gpt-3.5-turbo"
)

FineTuningJob(id='ftjob-knrobPPJMrpqmHH6132vec0k', created_at=1714320041, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-dStTSLOVLNSFy6iYbaKpbade', result_files=[], seed=712418011, status='validating_files', trained_tokens=None, training_file='file-Yue5dJCmzEb6emFnJwAMn9yB', validation_file='file-SMCZ1qrxfy9jT94xKyjAPDtW', integrations=[], user_provided_suffix=None, estimated_finish=None)

In [ ]:
response = client.fine_tuning.jobs.create(
  training_file="file-7baXAHqNdqyXZjOrS8KleWIR",
  validation_file='file-SMCZ1qrxfy9jT94xKyjAPDtW',
  model="gpt-3.5-turbo",
  hyperparameters={
    "n_epochs": 15,
	"batch_size": 32,
	"learning_rate_multiplier": 0.1
  }
)
job_id = response.id
status = response.status

print(f'Fine-tunning model with jobID: {job_id}.')
print(f"Training Response: {response}")
print(f"Training Status: {status}")

Fine-tunning model with jobID: ftjob-20MDkvkT5VYb5u0VHaE0ZjCw.
Training Response: FineTuningJob(id='ftjob-20MDkvkT5VYb5u0VHaE0ZjCw', created_at=1714320108, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=15, batch_size=32, learning_rate_multiplier=0.1), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-dStTSLOVLNSFy6iYbaKpbade', result_files=[], seed=1611691271, status='validating_files', trained_tokens=None, training_file='file-7baXAHqNdqyXZjOrS8KleWIR', validation_file='file-SMCZ1qrxfy9jT94xKyjAPDtW', integrations=[], user_provided_suffix=None, estimated_finish=None)
Training Status: validating_files


In [ ]:
for i in client.fine_tuning.jobs.list(limit=1):
    print(i)

FineTuningJob(id='ftjob-20MDkvkT5VYb5u0VHaE0ZjCw', created_at=1714320108, error=Error(code='invalid_training_file', message='The job failed due to an invalid training file. Invalid file format. Input file file-7baXAHqNdqyXZjOrS8KleWIR is in the prompt-completion format, but the specified model gpt-3.5-turbo-0125 is a chat model and requires chat-formatted data. See https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset for details.', param='training_file'), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=15, batch_size=32, learning_rate_multiplier=0.1), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-dStTSLOVLNSFy6iYbaKpbade', result_files=[], seed=1611691271, status='failed', trained_tokens=None, training_file='file-7baXAHqNdqyXZjOrS8KleWIR', validation_file='file-SMCZ1qrxfy9jT94xKyjAPDtW', integrations=[], user_provided_suffix=None, estimated_finish=None)
FineTuningJob(id='ftjob-knrobPPJMrpqmHH6132vec0k'

# Fine-Tuned Model Evaluation

In [ ]:
completion = client.chat.completions.create(
  model="ft:gpt-3.5-turbo-0125:personal::9IkvFJkJ",
  messages=[
    {"role": "system", "content": "```Listen up! My name is Debbi and I'm here to raise funds for my brother Donald Webb. Let me tell you something that will make your blood boil - he's only 51 years old and back in 2016, he was hit with a barrage of strokes that robbed him of his motor skills and memories. And if that wasn't enough, now his kidneys are failing and he's on the fast track to dialysis. As if that's not infuriating already, he's also battling high blood pressure and diabetes. He's stuck in the hospital for who knows how long, and every single day there costs a whopping $450 just to keep his bed at The Regency at Shelby in Shelby Township Mi, his care facility. We need all the help we can get to ensure he gets the care he deserves.```"},
    {"role": "user", "content": "What's the emotion of this sentence "}
  ]
)
print(completion.choices[0].message)


ChatCompletionMessage(content='The emotion conveyed in this passage is a mix of sadness, frustration, and urgency. It highlights the immense challenges faced by Donald Webb and the urgency of raising funds to support his healthcare needs.', role='assistant', function_call=None, tool_calls=None)


In [ ]:
completion = client.chat.completions.create(
  model="ft:gpt-3.5-turbo-0125:personal::9IkvFJkJ",
  messages=[
    {"role": "system", "content": "```Love enveloped me in its warm embrace as another seizure tried to take me down. In the midst of darkness, I was surrounded by the unwavering love of my family. Their presence, their strength, their unwavering support carried me through the toughest of times. My mother, the rock of our family, stood by my side, her love shining brighter than any fear or doubt. Her selflessness, her positivity, her resilience, all a testament to the power of a mother's love. I am forever grateful for her, for without her, I would be lost in a sea of uncertainty. Thank you, Mom, for being my guiding light, my source of love and strength.```"},
    {"role": "user", "content": "What's the emotion of this sentence "}
  ]
)
print(completion.choices[0].message)



ChatCompletionMessage(content='The emotion of this passage is gratitude and love.', role='assistant', function_call=None, tool_calls=None)


In [ ]:
completion = client.chat.completions.create(
  model="ft:gpt-3.5-turbo-0125:personal::9IkvFJkJ",
  messages=[
    {"role": "system", "content": "Out of nowhere, my blood pressure surged on Friday, sending me rushing to the ER. After being transferred to Dallas, I received a terrifying diagnosis of severe preeclampsia. I'm facing a long stay until Greyson arrives, possibly as early as 34 weeks. Not only will he be born prematurely, but he also has a heart defect requiring surgery. This unexpected turn has left us reeling, with 3 kids at home needing care and bills piling up. Joseph is juggling meals and trips between Dallas and Tyler. Any support during this frightening time would mean the world to us as we confront these daunting challenges."},
    {"role": "user", "content": "What's the emotion of this sentence "}
  ]
)
print(completion.choices[0].message)



ChatCompletionMessage(content='The emotion conveyed in this message is one of fear, uncertainty, and vulnerability. The unexpected and serious diagnosis of severe preeclampsia, the premature birth of the baby, and the additional complication of a heart defect requiring surgery have left the family reeling and facing daunting challenges. The mention of bills piling up and the struggle of juggling care for three children at home while also being in the hospital adds to the sense of stress and overwhelm. Overall, the tone is one of needing support and facing a difficult and frightening situation.', role='assistant', function_call=None, tool_calls=None)


In [ ]:
completion = client.chat.completions.create(
  model="ft:gpt-3.5-turbo-0125:personal::9IkvFJkJ",
  messages=[
    {"role": "system", "content": "Hello everyone. I am overwhelmed with gratitude for the love and support I have received during this challenging time. For the past 20 months, I have been facing mysterious medical issues that have led to numerous tests, procedures, and medications. While my health insurance has been helpful, the bills are starting to add up. I am humbly reaching out to ask for your help, not out of desperation, but out of love and appreciation for the community that surrounds me. Your generosity, no matter how big or small, will mean the world to me. Thank you for being a beacon of love and light in my life."},
    {"role": "user", "content": "What's the emotion of this sentence "}
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content='The emotion conveyed in the sentence is gratitude.', role='assistant', function_call=None, tool_calls=None)


In [ ]:
completion = client.chat.completions.create(
  model="ft:gpt-3.5-turbo-0125:personal::9IkvFJkJ",
  messages=[
    {"role": "system", "content": "Hello everyone, it pains me to write this message as I am in desperate need of help. My father recently passed away on 10/29, and we are struggling to afford his cremation. With my mom being the sole provider for me and my younger siblings, we are unable to give my father the proper farewell he deserves. The thought of not being able to provide him with a dignified resting place fills me with overwhelming sadness. Despite the heartache, I am grateful for the support and love we have received during this difficult time. We are now trying to raise funds for his cremation and other expenses, including a piece of jewelry to keep his memory close to our hearts. Your help in this time of sorrow would mean the world to us. Thank you for your kindness."},
    {"role": "user", "content": "What's the emotion of this sentence "}
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content='The emotion conveyed in the message is sadness and desperation.', role='assistant', function_call=None, tool_calls=None)


In [ ]:
completion = client.chat.completions.create(
  model="ft:gpt-3.5-turbo-0125:personal::9IkvFJkJ",
  messages=[
    {"role": "system", "content": "My wife Janice was recently diagnosed with Acute Promyelocytic Leukemia, and now we're stuck dealing with the financial burden of her hospital stay. The expenses keep piling up - ambulance costs, hospital bills, mortgage payments, childcare for our daughters - it's all too much to handle. We shouldn't have to rely on a GoFundMe campaign to get by. It's infuriating that we're in this situation, and I can't help but feel angry at the unfairness of it all. We're supposed to believe that praying to Yeshua is the answer, but right now, all I feel is anger at the hand we've been dealt."},
    {"role": "user", "content": "What's the emotion of this sentence "}
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="The primary emotion conveyed in this passage is anger. The speaker expresses frustration and resentment at the financial burden and hardships they are facing due to their wife's illness.", role='assistant', function_call=None, tool_calls=None)


## Reference

The above is developed based on materials from [1], [2], [3], and [4]



[1] “LLMs and Marketing Narratives” (work in progress), Samsun Knight and Liu Liu

[2] https://stanford-cs324.github.io/winter2022/projects/CS324_P1.pdf

[3] Li, Zaijing, Gongwei Chen, Rui Shao, Dongmei Jiang, and Liqiang Nie. "Enhancing the Emotional Generation Capability of Large Language Models via Emotional Chain-of-Thought." arXiv preprint arXiv:2401.06836 (2024).

[4] Shu, Lei, Liangchen Luo, Jayakumar Hoskere, Yun Zhu, Yinxiao Liu, Simon Tong, Jindong Chen, and Lei Meng. "Rewritelm: An instruction-tuned large language model for text rewriting." In Proceedings of the AAAI Conference on Artificial Intelligence, vol. 38, no. 17, pp. 18970-18980. 2024.
